In [1]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.nn.parameter import Parameter
import sys

In [2]:
# import dataset class 
dataset_folder_path = '/Users/ebrahimfeghhi/NCEL/rnn_modeling/datasets/'
sys.path.insert(1, dataset_folder_path)
from CFD_class import CFD

In [3]:
# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
class NeuroRNN(nn.Module):
    
    def __init__(self, input_size, output_dim, alpha_r, alpha_s, nonlinearity, hidden_size, bias):
        
        super(NeuroRNN, self).__init__()
        self.input_size = input_size
        self.nonlinearity = nonlinearity
        self.hidden_size = hidden_size 
        self.alpha_r = alpha_r
        self.alpha_s = alpha_s
        self.bias = bias 
        self.output_dim = output_dim
        self.Win = Parameter(torch.Tensor(hidden_size, input_size))
        self.Win.requires_grad = True 
        self.Wrec = Parameter(torch.Tensor(hidden_size, hidden_size))
        self.Wrec.requires_grad = True 
        self.Wout = Parameter(torch.Tensor(output_dim, hidden_size))
        self.Wout.requires_grad = True
        
        if bias:
            self.bin = Parameter(torch.Tensor(hidden_size, hidden_size))
            self.brec = Parameter(torch.Tensor(hidden_size))
            
        # init weights 
        nn.init.orthogonal_(self.Win)
        nn.init.orthogonal_(self.Wrec)
        nn.init.uniform_(self.Wout)
        
            
    def nonlin(self, inp):
            
        if self.nonlinearity == 'relu':
            nn.ReLU(inp)
            return inp
        else:
            inp = torch.tanh(inp)
            return inp

    def forward(self, x):
        
        # x shape: (batch_size, seq_len, input_size)
        I = torch.zeros(self.hidden_size, x.size(0)).to(device) # (hidden_size, batch_size)
        r = torch.zeros(self.hidden_size, x.size(0)).to(device)
        out = torch.zeros(batch_size, x.size(1), self.output_dim)
        
        r_total = torch.zeros(batch_size, x.size(1))
        r_total.requires_grad = True 
        
        for t in range(x.size(1)):
            B_t = x[:, t].T # all batched data at t (input_size, batch_size)
            I = (1-self.alpha_s)*I + self.alpha_s*(torch.mm(self.Wrec,r) + torch.mm(self.Win, B_t))
            r = (1-self.alpha_r)*r + self.alpha_r*(self.nonlin(I))
            r_total[:, t] = r.T
            out[:, t, :] = torch.mm(self.Wout, r).T
    
        return out, r_total

In [10]:
learning_rate = 5e-5
num_epochs = 10
sequence_length = 10
batch_size = 32

model = NeuroRNN(1,2,1,1,'relu',20,False)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# load data 
training_set = CFD(2400, sequence_length, [.1,.9], 0, 5, 0, 5, 0)

trainloader = DataLoader(training_set, batch_size=batch_size,
        shuffle=True)



In [11]:
for epoch in range(num_epochs):

    for idx, (data, targets) in enumerate(trainloader):
        
        
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores, r_total = model(data)
        l1_loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        
        l1_loss.backward()
    
        optimizer.step()

RuntimeError: expand(torch.FloatTensor{[32, 20]}, size=[32]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)